In [1]:
import sys
from itertools import count
from torch import autograd
from torch_geometric.utils import dense_to_sparse
import copy
from collections import defaultdict

sys.path.append('../../')

from src.models.gcn import *
from src.utils.datasets import *
from src.models.trainable import *
from src.attacks.meta_pgd import *
from src.attacks.greedy_gd import *
print(sys.executable)

/home/niyati/miniconda3/envs/ersp_v2/bin/python


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# dataset_directory = "../Cora"
cora_dataset = Planetoid(root='', name='Citeseer')
data = cora_dataset[0].to(device)
print(data)

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [4]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [64]).to(device)

In [5]:
model.reset_parameters()
train = Trainable(model)
train.fit(data, 200)

Epoch 0, Train Loss - 1.80226731300354, Val Loss - 1.6550486087799072, Val Accuracy - 0.598
Epoch 20, Train Loss - 0.00882615428417921, Val Loss - 1.220491886138916, Val Accuracy - 0.686
Epoch 40, Train Loss - 0.007291069719940424, Val Loss - 1.136745572090149, Val Accuracy - 0.678
Epoch 60, Train Loss - 0.011892858892679214, Val Loss - 1.077873945236206, Val Accuracy - 0.686
Epoch 80, Train Loss - 0.014267326332628727, Val Loss - 1.0761858224868774, Val Accuracy - 0.673
Epoch 100, Train Loss - 0.009532494470477104, Val Loss - 1.054239273071289, Val Accuracy - 0.689
Epoch 120, Train Loss - 0.007554575335234404, Val Loss - 1.0511032342910767, Val Accuracy - 0.69
Epoch 140, Train Loss - 0.007631202694028616, Val Loss - 1.0962022542953491, Val Accuracy - 0.678
Epoch 160, Train Loss - 0.011228954419493675, Val Loss - 1.1235802173614502, Val Accuracy - 0.679
Epoch 180, Train Loss - 0.006396390963345766, Val Loss - 1.0880025625228882, Val Accuracy - 0.689
Epoch 200, Train Loss - 0.0063984384

In [6]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.676
Initial Loss: 1.0949546098709106


In [14]:
attacker.num_budgets

NameError: name 'attacker' is not defined

In [7]:
from collections import defaultdict 
from torch_geometric.utils import to_networkx

# Run MetaPGD attack
ptb_rate = 1.0  # 100% perturbation rate
attacker = MetaPGD(data, device=device)
attacker.setup_surrogate(model,
                         labeled_nodes=data.train_mask,
                         unlabeled_nodes=data.test_mask,
                         lambda_=0.)
attacker.reset_parameters(seed=42)
attacker.reset()

best_loss, best_pert = attacker.attack(ptb_rate, base_lr=0.01, grad_clip=1, iterations=400)

Running PGD Attack...:   0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, device='cuda:0')
4552 tensor(4543, de

In [8]:
# Extract edges from the perturbation matrix
edges_added = []
modified_adj = attacker.get_perturbed_adj(best_pert).cpu().numpy()
original_adj = attacker.adj.cpu().numpy()

# Find added edges (in the upper triangle to avoid duplicates since graph is undirected)
for i in range(data.num_nodes):
    for j in range(i+1, data.num_nodes):  # Upper triangle only
        if modified_adj[i, j] > 0 and original_adj[i, j] == 0:
            edges_added.append((i, j))

print(f"Total edges added: {len(edges_added)}")

Total edges added: 4543


In [9]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.20
budget = int(ptb_rate * initial_edge_count)
print(f"initial edges: {initial_edge_count}")
print(f"Budget: {budget}")

initial edges: 2276
Budget: 455


In [15]:
print("Unique perturbed edges (undirected):", len(set(frozenset(e) for e in all_perturbed_edges)))

Unique perturbed edges (undirected): 1814


In [12]:
attacker.num_budgets

5278

In [13]:
attacker.adj_changes.sum()

tensor(10154., device='cuda:0')

In [10]:
from torch_geometric.utils import dense_to_sparse, to_networkx, from_networkx
import networkx as nx

In [11]:
print(len(edges_added))
print(budget)

4543
455


In [12]:
def two_phase_attack(split):
    diff_threshold = 0.01
    first_phase_edges = int(budget * split)
    second_phase_percent = ptb_rate * (1 - split) * 0.5
    print(f"\n--- Running split: {split} ---")
    print(f"Second phase perturbation rate: {second_phase_percent:.4f}")

    phase1_accuracies = []
    phase2_accuracies = []

    G = to_networkx(data, to_undirected=True)
    data_copy = copy.copy(data)

    i, j = 0, 0  # i - edges successfully added, j - index in list

    # === Phase 1 ===
    while i < first_phase_edges:
        if j >= len(edges_added):
            print("Ran out of candidate edges in Phase 1. Moving to Phase 2.")
            break

        u, v = edges_added[j]
        G.add_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)

        if modified_accuracy == initial_accuracy:
            i += 1
            phase1_accuracies.append(modified_accuracy)
        else:
            G.remove_edge(u, v)

        j += 1

    print(f"Phase 1: Added {i} edges out of requested {first_phase_edges}.")

    # === Phase 2 ===
    modified_data = from_networkx(G).to(device)
    modified_data.x = data.x 
    modified_data.y = data.y 
    modified_data.train_mask = data.train_mask
    modified_data.test_mask = data.test_mask

    attacker = Metattack(modified_data, device=device)
    attacker.setup_surrogate(model,
                             labeled_nodes=data.train_mask,
                             unlabeled_nodes=data.test_mask, 
                             lambda_=0.)
    attacker.reset()
    attacker.attack(second_phase_percent)

    degs = defaultdict(tuple)
    for k, v in attacker._added_edges.items():
        degs[v] = (k, True)
    for k, v in attacker._removed_edges.items():
        degs[v] = (k, False)

    for _, second in degs.items():
        u, v = second[0]
        if second[1]:
            G.add_edge(u, v)
        else:
            G.remove_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)
        phase2_accuracies.append(modified_accuracy)

    # === Final Reporting ===
    final_accuracy = phase2_accuracies[-1] if phase2_accuracies else (
        phase1_accuracies[-1] if phase1_accuracies else initial_accuracy)
    accuracy_drop = initial_accuracy - final_accuracy

    print(f"Final Accuracy: {final_accuracy:.4f}")
    print(f"Accuracy Drop: {accuracy_drop:.4f}")

    return {
        "split": split,
        "phase1_added": i,
        "phase1_accuracies": phase1_accuracies,
        "phase2_accuracies": phase2_accuracies,
        "final_accuracy": final_accuracy,
        "accuracy_drop": accuracy_drop
    }


In [13]:
splits = [0, 0.5]
split_dic = defaultdict(list)

In [14]:
for s in splits:
    print(s)
    split_dic[s] = two_phase_attack(s)

0

--- Running split: 0 ---
Second phase perturbation rate: 0.1000
Phase 1: Added 0 edges out of requested 0.


Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Final Accuracy: 0.6590
Accuracy Drop: 0.0170
0.5

--- Running split: 0.5 ---
Second phase perturbation rate: 0.0500
Phase 1: Added 227 edges out of requested 227.


Peturbing graph...:   0%|          | 0/238 [00:00<?, ?it/s]

Final Accuracy: 0.6710
Accuracy Drop: 0.0050
